## Demo for project 1

In [ ]:
import torch
import torchvision

### Define dataset and dataloader

In [ ]:
%load_ext autoreload
%autoreload 2

from dataset import load_dataset

trainset, valset = load_dataset(train=True,is_vert_flip = False,is_rotate = True,is_translate = True,is_color_jitter = False,is_DG=False)
testset = load_dataset(train=False)

batch_size = 4

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

### Check dataset and dataloader

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    # The data need to be normalized and unnormalized to keep the same
    # img = img / 255
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# get some random training images
dataiter = iter(trainloader)
images, targets = next(dataiter)
# show images
imshow(torchvision.utils.make_grid(images))
# print(names)
for i in range(batch_size):
    target = targets.numpy()[i,:,:]
    plt.subplot(1,batch_size,i+1)
    plt.imshow(target,cmap='gray')


### Q1: Domain generalization example.

In [ ]:
from dataset import domain_generization, load_dataset
trainset_temp, _ = load_dataset(train=True,is_vert_flip = True,is_rotate = True,is_translate = True,is_color_jitter = False,is_DG=True)
# trainset_temp, _ = load_dataset(train=True,is_vert_flip = False,is_rotate = False,is_translate = False,is_color_jitter = False,is_DG=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
original_image, _ = trainset_temp.__getitem__(5) # Use the first image in training set.
original_image = original_image / 2 + 0.5
original_image = original_image.numpy()
scaling_factor = 0.01 # 替换低频区域所占大小
ratio = 1 #替换区域中目标域图片的幅度比重 
num_generalized=10
domains = ['domain1','domain2','domain3'] # 可选：'random'（默认随机选择domain）, 'domain1', 'domain2', 'domain3'.
original_image_freq = np.fft.fftshift(np.fft.fft2(original_image,axes=(-2,-1)),axes=(-2,-1))
original_image_freq = np.log(np.abs(original_image_freq))/np.max(np.log(np.abs(original_image_freq))) # log and normalize
plt.figure(figsize=[4,2])
plt.subplot(1,2,1), plt.imshow(np.transpose(original_image, (1, 2, 0))), plt.xlabel('Raw Image.')
plt.subplot(1,2,2), plt.imshow(np.transpose(original_image_freq, (1, 2, 0))), plt.xlabel('Raw Image (frequency).')
dg_outputs_domains = []
for domain in domains:
    dg_outputs, dg_fre_outputs= np.array(domain_generization(original_image,scaling_factor, ratio,num_generalized,domain)) # 输出是一个float, 因为计算傅里叶变换的时候应该用float提高精度
    fig, axs = plt.subplots(2, num_generalized, figsize=(2*num_generalized, 2*2), layout="constrained")
    for i in range(num_generalized):
        dg_output = np.real(dg_outputs[i])
        dg_fre_output = dg_fre_outputs[i]
        dg_fre_output = np.log(np.abs(dg_fre_output))/np.max(np.log(np.abs(dg_fre_output))) # log and normalize
        axs[0,i].imshow(np.clip(np.transpose(dg_output, (1, 2, 0)),0,1)) # 转回int才可以直接imshow
        axs[0,i].set_xlabel('After DG (image).')
        axs[1,i].imshow(np.clip(np.transpose(dg_fre_output,(1,2,0)),0,1),cmap='gray')
        axs[1,i].set_xlabel('After DG (frequency).')
    plt.suptitle('Domain generalization example: {}.'.format(domain))
    dg_outputs_domains.append(dg_outputs)
dg_outputs = np.asarray(dg_outputs)

In [ ]:
from dis_rep import norm_dist,CS_dist,intra_cluster_distance,inter_cluster_diatance


### Define a model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp

net = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)
# print(net)

### Define loss function and optimizer

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train network

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, target) in enumerate(trainloader):
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimizeo 
        # change the data to float type
        outputs = net(inputs)
        # outputs = torch.argmax(outputs, dim=1)
        # print((outputs.shape))
        # print(target.to(torch.float32).dtype)
        
        loss = criterion(outputs,target.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

print('Finished Training')



In [ ]:
# show last valiadated result
import matplotlib.pyplot as plt
import numpy as np

input = inputs.numpy()[-1]
input = input/2 + 0.5
output_np = torch.argmax(outputs[-1], dim=0).cpu().numpy()

plt.subplot(1,3,1)
plt.imshow(np.transpose(input,(1,2,0)))
plt.subplot(1,3,2)
plt.imshow(target[-1,:,:],cmap='gray')
plt.subplot(1,3,3)
plt.imshow(output_np,cmap='gray')
plt.show()

### Save model

In [ ]:
PATH = './net.pth'
torch.save(net.state_dict(), PATH)

### Test your model performance

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

nclasses = 2

net = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=nclasses,                      # model output channels (number of classes in your dataset)
)


net.load_state_dict(torch.load(PATH,map_location='cpu'))
dscs = []

with torch.no_grad():
    for data in testloader:
        images, targets, names = data
        
        outputs = net(images.to(torch.float32))
        
        for idx, name in enumerate(names):
            output_np = torch.argmax(outputs[idx], dim=0).cpu().numpy()
            binary_output = np.array(output_np)
            target_np = targets[idx].cpu().numpy().astype(np.uint8)
            
            target_1d = np.reshape(target_np, (-1, 1))
            pred_1d = np.reshape(binary_output, (-1, 1))
            accuracy = accuracy_score(target_1d, pred_1d)
            if nclasses == 2:
                dsc = f1_score(target_1d, pred_1d)
            else:
                dsc = f1_score(target_1d, pred_1d,average='micro')
            dscs.append(dsc)

dsc_test = np.mean(dscs)
print('Dsc of test set:', dsc_test)

In [ ]:
# show last predicted result
import matplotlib.pyplot as plt
image = images.numpy()[-1]
output = outputs.numpy()[-1,:,:,:]
output = output/2 + 0.5
plt.subplot(1,3,1)
plt.imshow(np.transpose(image,(1,2,0)))
plt.subplot(1,3,2)
plt.imshow(target_np,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(binary_output,cmap='gray')
plt.show()